In [1]:
import torch
from utils import MLP

In [2]:
raw_names = open("./names.txt").read()
names = raw_names.split('\n')

In [3]:
X = []
Y = []
block_size = 3

letters_n = {v:i+1 for i,v in enumerate(sorted(list(set(raw_names.replace('\n','')))))}
letters_n['.'] = 0
n_letters = {v:i for i,v in letters_n.items()}

for name in names:
    context = [0] * block_size
    for char in name:
        n_char = letters_n[char]
        X.append(context)
        Y.append(n_char)
        
        context = context[1:] + [n_char]

X = torch.Tensor(X)
Y = torch.Tensor(Y)

X.dtype, Y.dtype

(torch.float32, torch.float32)

In [64]:
gen = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,3), generator=gen)
embeddings = C[X.type(dtype=torch.int64)]
W1 = torch.randn((9, 100), generator=gen)
b1 = torch.randn(100, generator=gen)
W2 = torch.randn((100, 27), generator=gen)
b2 = torch.randn(27, generator=gen)
epsilon = 0.01

In [73]:
hypothesis = torch.tanh(embeddings.view(-1,9) @ W1 + b1)                                                # First Hidden Layer 
logits = hypothesis @ W2 + b2                                                                           # Logits
counts = logits.exp()                                                                                   # Output Layer
prob = counts / counts.sum(1, keepdim=True)                                                             # Softmax
loss = -prob[torch.arange(embeddings.shape[0]).type(torch.int64), Y.type(torch.int64)].log().mean()     # Loss